In [1]:
%pwd

'c:\\Users\\NARINDER\\Desktop\\Local Chatbot\\research'

In [2]:
import os
os.chdir("..")  # Change to the parent directory

In [3]:
%pwd

'c:\\Users\\NARINDER\\Desktop\\Local Chatbot'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    unzip_dir: Path

In [14]:
from src.chatbot.constants import *
from src.chatbot.utils.common import read_yaml, create_directory

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directory([self.config.artifact_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directory([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_url=config.source_url,
            unzip_dir=Path(config.unzip_dir)
        )
        return data_ingestion_config

In [ ]:
from kaggle import api
from pathlib import Path
from src.chatbot import logger

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self) -> Path:
        logger.info(f"Downloading file from {self.config.source_url} to {self.config.root_dir}")
        api.dataset_download_files(dataset=self.config.source_url, path=self.config.root_dir, unzip=True)
        
    def initiate_data_ingestion(self) -> Path:
        self.download_data()
        return self.config.unzip_dir

In [28]:
from src.chatbot import logger
from kaggle import api

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.initiate_data_ingestion()

except Exception as e:
    logger.exception(e)
    raise e

[2025-10-21 16:34:42,714: INFO: common]: yaml file: config\config.yaml loaded successfully
[2025-10-21 16:34:42,718: INFO: common]: yaml file: params.yaml loaded successfully
[2025-10-21 16:34:42,722: INFO: common]: Directory created at: artifacts
[2025-10-21 16:34:42,725: INFO: common]: Directory created at: artifacts/data_ingestion
[2025-10-21 16:34:42,727: INFO: 301372740]: Downloading file from niraliivaghani/chatbot-dataset to artifacts\data_ingestion
Dataset URL: https://www.kaggle.com/datasets/niraliivaghani/chatbot-dataset
